In [18]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from tw_sentiment import preprocess_tweet, load_roberta_model_tokenizer, analyze_tweet

In [31]:
data = {
    'monero' : {
        'query' :  'monero lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'bitcoin' : {
        'query' :  'bitcoin lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'dogecoin' : {
        'query' :  'dogecoin lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'ethereum' : {
        'query' :  'ethereum lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'solana' : {
        'query' :  'solana lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'cardano' : {
        'query' :  'ethereum lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
    'radix' : {
        'query' :  'radix lang:en since:2023-06-10',
        'dataframe' : None,
        'optimism_score' : 0,
        },
}

In [32]:
tokenizer, model = load_roberta_model_tokenizer()

In [33]:

limit = 100
for key in data:
    query = data[key]['query']
    print(query)
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) == limit:
            break
        else:
            tweets.append([tweet.date, tweet.user.username , tweet.rawContent])
    data[key]['dataframe'] = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
    

monero lang:en since:2023-06-10
bitcoin lang:en since:2023-06-10


Stopping after 20 empty pages


dogecoin lang:en since:2023-06-10


Unavailable user in card on tweet 1670680506969608195
User 933016351756742656 not found in user refs in card on tweet 1670680506969608195
Unavailable user in card on tweet 1670679569031131136
User 933016351756742656 not found in user refs in card on tweet 1670679569031131136
Unavailable user in card on tweet 1670679433571606530
User 933016351756742656 not found in user refs in card on tweet 1670679433571606530


ethereum lang:en since:2023-06-10
solana lang:en since:2023-06-10
ethereum lang:en since:2023-06-10


In [41]:
for key in data:
    df = data[key]['dataframe']
    df['processed_tweets'] = df['Tweet'].apply(lambda x: preprocess_tweet(x))
    df['optimism_score'] = df['processed_tweets'].apply(lambda x: analyze_tweet(tokenizer, model, x))
    data[key]['optimism_score'] = np.sum(df['optimism_score'])
    # Remove timezone from columns - otherwise there are problems with exporting the dataframe to Excel
    df['Date'] = df['Date'].dt.tz_localize(None)

In [42]:
def print_scores(data):
    for key in data:
        print(f"{key}: {data[key]['optimism_score']}")
print_scores(data)

monero: 15.119610434165224
bitcoin: 1.8586253740359098
dogecoin: 24.44772095803637
ethereum: 35.8623839856009
solana: 29.187380094837863
cardano: 35.7318348690751


In [43]:
data['ethereum']['dataframe']

,Date,User,Tweet,processed_tweets,optimism_score
0,2023-06-19 06:33:42,Zayts_ETHs,"gn fam 🧡, the last $LOYAL airdrop is now live ...","gn fam 🧡, the last $LOYAL airdrop is now live ...",0.099552
1,2023-06-19 06:33:36,yeti_0x,@danieleripoll @Shamik04660398 @hodlmee @thepo...,"@user @user @user @user Misspoke, meant not a ...",-0.170218
2,2023-06-19 06:33:34,Zayts_ETHs,"gn fam 🧡, the last $LOYAL airdrop is now live ...","gn fam 🧡, the last $LOYAL airdrop is now live ...",0.099552
3,2023-06-19 06:33:21,SygnalAi,SYGNAL - Daily Ethereum signal. June 19th 2023...,SYGNAL - Daily Ethereum signal. June 19th 2023...,0.092479
4,2023-06-19 06:33:20,OtaniKoich60509,Layer 1.5 solutions can involve the loading an...,Layer 1.5 solutions can involve the loading an...,0.253265
...,...,...,...,...,...
95,2023-06-19 06:12:12,DavieHenryGravy,Beware of the Sleepdrop Crypto Scam - Protect ...,Beware of the Sleepdrop Crypto Scam - Protect ...,-0.455761
96,2023-06-19 06:12:06,NoticCrypto,Which crypto project do you trust the most?\n\...,Which crypto project do you trust the most?\n\...,0.100340
97,2023-06-19 06:12:01,CryptoEgod,🎁GIVEAWAY 🎁\n$5 Ethereum #Giveaway \nEnd in ...,🎁GIVEAWAY 🎁\n$5 Ethereum #Giveaway in • 48...,0.552495
98,2023-06-19 06:11:58,levi28473,"@yourcryptodj $OMNOM ,the most charming dinosa...","@user $OMNOM ,the most charming dinosaur on th...",0.970039


In [44]:
data['ethereum']['dataframe'].to_excel('ethereum.xlsx')